<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Agentic AI Workflows - Week 2</center></font>

<center><p float="center">
  <img src="https://images.pexels.com/photos/33910465/pexels-photo-33910465.jpeg" width=720></a>
<center><font size=6>Greatglobe Logistics Assistant: Automating Product Compliance and Customs Guidance</center></font>

#**Problem Statement**

## Business Context

Greatglobe Logistics is a global supply chain company that manages the shipment of more than 500 products for 20-30 client companies across multiple domains. Each shipment may require compliance with international trade regulations, including import/export documentation, customs duties, and payment procedures. For logistics managers and supply chain teams, gathering accurate documentation and understanding payment obligations for each shipment is time-consuming and prone to errors. Automating this process using data and intelligent agents can greatly improve operational efficiency.

Currently, retrieving product details and corresponding trade compliance requirements is a manual, fragmented process:

- Product information is stored in internal databases (Product_ID, category, cost, etc.).
- Compliance information such as required import/export documents, duty payment methods, and obligations must be researched online, which is slow and inconsistent.

This makes it difficult for logistics teams to quickly verify shipments, especially when handling multiple clients across different countries.

##  Objective

The goal of this project is to build an interactive logistics assistant that:

- Retrieves product details from a SQL database based on a user-provided Product_ID.
- Uses a web agent to fetch up-to-date import/export documents and payment methods for the product’s HS/HSN code, based on source and destination countries.
- Presents the information in a structured format, highlighting both required documentation and any additional compliance information if available.

## Data Description

The database contains product-level information that includes:

- Product_ID - Unique identifier for each product.
- Product_Name - Name of the product.
- Category - Broad classification of the product (e.g., Electronics, Textiles, Pharmaceuticals).
- HSN_Code - Harmonized System of Nomenclature code for customs classification.


#**Installing and Importing Necessary Libraries**

In [ ]:
!pip install -q tavily-python==0.7.12 \
                langchain-community==0.3.29

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


**Note:**

- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [ ]:
import sqlite3                                                       #Imports Python’s built-in SQLite library to create and interact with SQLite databases.
import pandas as pd                                                  #Imports Pandas for data manipulation and reading CSV files.
from openai import OpenAI                                            #Imports OpenAI for parsing search results (Optional)
import json                                                          #Imports Json for handling search results
import os
from langchain.agents import create_sql_agent
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import json
from tavily import TavilyClient                                      #Tavily Search API, which performs real-time web searches.

In [ ]:
# Load the JSON file and extract values
file_name = 'config.json'
with open(file_name, 'r') as file:
    config = json.load(file)
    OPENAI_API_KEY = config.get("OPENAI_API_KEY") # Loading the API Key
    OPENAI_API_BASE = config.get("OPENAI_API_BASE") # Loading the API Base Url


# Storing API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE

#**SQL Agent**

SQL agents help retrieve the product category based on the given product ID.

In [ ]:
# Initialize the SQLDatabase object
db = SQLDatabase.from_uri("sqlite://///content/greatglobe.db")

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # You might need to set your OpenAI API key

# Initialize a SQL agent to interact with the database using the LLM
sql_agent = create_sql_agent(
    llm,
    db=db,
    agent_type="openai-tools",
    verbose=False
)

/tmp/ipython-input-294495338.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) # You might need to set your OpenAI API key


In [ ]:
query = f"Fetch the category with product_id P1016"


# Execute the query using the SQL agent and store the output
output = sql_agent.invoke(query)

# Display the retrieved records
output['output']

'The category for the product with product_id P1016 is "Pharmaceuticals".'

#**Tavily Client**

**Instruction:**
To create a Tavily API key, please follow these steps:

1. Click the link provided to go to the [Tavily](https://app.tavily.com/home) website.
2. Sign in to the Tavily website.
3. Generate a new API key for your use.
4. Copy the API key and store it securely for use in your project.

In [ ]:
# Replace with your Tavily API Key
TAVILY_API_KEY = "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"

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Build your query dynamically based on product and route
query = f"List required import/export documents and accepted payment methods for customs clearance (e.g., cash, cheque, online, duty payment portal) for goods of FMCG category shipping from USA to India"

# Call Tavily API
try:
    response = tavily_client.search(query)
    print("=== Tavily Response ===")
    print(response)
except Exception as e:
    print("Error fetching data from Tavily API:", e)


=== Tavily Response ===
{'query': 'List required import/export documents and accepted payment methods for customs clearance (e.g., cash, cheque, online, duty payment portal) for goods of FMCG category shipping from USA to India', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://artemusgroupusa.com/custom-clearance-documents/', 'title': 'Custom Clearance Documents Required In USA, India, & Japan', 'content': 'The mandatory documents for import typically include a commercial invoice, bill of lading or airway bill, packing list, import license or permit', 'score': 0.65966886, 'raw_content': None}, {'url': 'https://abhyanshshipping.com/step-by-step-guide-to-customs-clearance-in-india-what-every-importer-should-know/', 'title': 'Custom Clearance in India: Step-by-Step Import Guide for 2025', 'content': '**Importing goods into India for the first time can feel overwhelming.** The customs clearance process involves multiple steps, paperwork, and complian

#**Logistic Assistant Agent**

## Search Compliance Info

In [ ]:
def search_compliance_info(shipment_details: str):
    """
    Uses Tavily API to search for required import/export documents and payment methods.
    """
    # Build the query for Tavily based on category, source, and destination
    tavily_query = f"required import/export documents and accepted payment methods for customs clearance for goods of following category and shipping source and dest{shipment_details}"
    tavily_response = tavily_client.search(tavily_query)
    return tavily_response["results"]

## Format Compliance Info

In [ ]:
def format_compliance_info(tavily_response: str):

    prompt = f"""
    You are a logistics assistant. Read the response throughly before generating a response. understand which countries are being refered to in the response.
    Extract structured information from the following Tavily API response.

    Requirements:
    1. List all required import/export documents clearly under "required_documents". Please ensure that the documents in the list are not repeated.
    2. If there is other important information (e.g., payment modes, duty rates, obligations, restrictions, licenses, etc.),
       list it under "additional_information".
    3. If no additional information is available, do not include the field.

    Tavily response:
    {tavily_response}
    """

    client = OpenAI()

    # Assuming client (OpenAI client) is initialized elsewhere
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    return completion.choices[0].message.content

## Create Agent

In [ ]:
# Define tools for the agent
tools = [
    Tool(
        name="Search Compliance Info",
        func=search_compliance_info,
        description="Use this tool to search for import/export compliance information using Tavily.",
    ),

     Tool(
        name="Format Compliance Info",
        func=format_compliance_info,
        description="Use this tool to format the raw Tavily search results into a structured output. Input should be the string output from the 'Search Compliance Info' tool."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

In [ ]:
def Logistic_Assistant_Agent():
    source = input("Enter Source Country: ")
    destination = input("Enter Destination Country: ")
    product_id = input("Enter Product ID: ")

    sql_query = f"Fetch the category for product id '{product_id}'"
    category_result = sql_agent.invoke(sql_query)
    category = category_result['output']
    category = category.replace("", "")

    # Pass structured input to the agent
    agent_input = f"Below are the details for the query Category: {category},Source: {source},Destination: {destination}"
    response = agent.invoke(agent_input)

    print("Compliance Info:")
    print(response['output'])


#**Sample Test Case**

## Test Case 1

- Origin : USA
- Destinantion : Germany
- Product ID : P1016

In [ ]:
Logistic_Assistant_Agent()

Enter Source Country: USA
Enter Destination Country: Germany
Enter Product ID: P1016
Compliance Info:
The import/export compliance information for pharmaceutical products from the USA to Germany includes required documents such as Entry Manifest, Application and Special Permit for Immediate Delivery, Entry summary for consumption, Estimated duties deposit, and Entry summary documentation. Additionally, compliance with U.S. regulations and requirements of the destination country is necessary, along with disclosure of basic information and updates in regulations for pharma and biotech companies in 2025.


## Test Case 2

- Origin : United Kingdom
- Destinantion : France
- Product ID : P1001

In [ ]:
Logistic_Assistant_Agent()

Enter Source Country: United Kingdom
Enter Destination Country: France
Enter Product ID: P1001
Compliance Info:
The required documents for shipping the product category "Automobile" from the United Kingdom to France include a Commercial Invoice, Road Waybill, and TIR Carnets. Additional information includes specific procedures for travellers entering or leaving France, availability of information in English for handling formalities, and guidelines provided by French Customs for Brexit preparations and successful imports.


# Conclusion

The proposed **Logistics Assistant Agent** offers a step forward in automating compliance checks and streamlining shipment verification for Greatglobe Logistics. By integrating structured product information from the internal SQL database with real-time compliance intelligence from external sources (via Tavily and OpenAI), the system reduces dependency on manual research and fragmented processes.

From a **business impact perspective**, this solution addresses three key challenges:

1. **Operational Efficiency** - Logistics managers and supply chain teams no longer need to manually search for country-specific compliance documents or duty payment methods. The assistant delivers structured, accurate, and timely information, accelerating decision-making and reducing turnaround time per shipment.

2. **Error Reduction and Compliance Assurance** - Trade compliance errors can be costly, leading to shipment delays, penalties, or reputational risks. Automating document retrieval and payment obligation identification minimizes human oversight and ensures that shipments meet international regulatory requirements consistently.

3. **Scalability Across Clients and Regions** - With Greatglobe handling 20–30 clients and over 500 products across multiple domains, the assistant is designed to scale. It adapts to different product categories and trade routes, making it a versatile tool that supports growth without proportional increases in compliance workload.

Overall, the solution enables efficient management of multiple products and clients across countries, streamlining global trade operations.


<font size = 6 color="#4682B4"><b> Power Ahead </font>
___